In [2]:
!pip uninstall -y transformers accelerate evaluate rouge_score
!pip install transformers[torch] accelerate -U
!pip install evaluate rouge_score
!pip install sacrebleu
!pip install meteor

Found existing installation: transformers 4.31.0
Uninstalling transformers-4.31.0:
  Successfully uninstalled transformers-4.31.0
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d839691f5708c3675c100e77a4ff2183d9a60c8b3ce0c9e56e33d875d5f46cee
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully bui

In [3]:
!pip install bert_score

In [4]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=79707d75807cd281fbdf61536b825d21998411c264cc523df93ffdaa6e38cd7d
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [5]:
import wandb
wandb.init(project ="team6")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import evaluate
import tensorflow as tf
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader


from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [8]:
df = pd.read_excel("/content/drive/MyDrive/AimlQA/aiml-qa-dev.xlsx")
df.head()

,question,answer1,answer2
0,Does the maximum value of 'n' in the n-gram mo...,The maximum value of n in an n-gram model is n...,The maximum value of 'n' in an n-gram model do...
1,How CNN works?,Convolutional Neural Networks (CNNs) use convo...,CNN is a type of neural network that is common...
2,How is NMT trained? Is it common to use pairs ...,Neural Machine Translation (NMT) is typically ...,"Yes, training NMT models involves using parall..."
3,What is the process of learning POS tags?,The process of learning POS tags involves trai...,POS tags are learned by training machine learn...
4,how to handle multi lingual situations in NLP?,Handling multilingual situations in NLP involv...,It involves techniques such as language identi...


In [9]:
list_of_references = []
for i in range(0, len(df)):
    list_of_references.append([df.iloc[i, 1], df.iloc[i, 2]])
print(len(list_of_references))
print(len(df))

80
80


In [10]:
train_file_path = "/content/drive/MyDrive/AimlQA/question-answer-train.txt"
eval_file_path = "/content/drive/MyDrive/AimlQA/question-answer-dev.txt"
model_name = 'gpt2'
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")
sacrebleu = evaluate.load("sacrebleu")
#meteor = evaluate.load('meteor')
output_dir = '/kaggle/working/result'
overwrite_output_dir = False
per_device_train_batch_size = 4
num_train_epochs = 10
save_steps = 10000

In [13]:
def load_dataset(file_path, tokenizer):
    dataset = LineByLineTextDataset(
                tokenizer=tokenizer,
                file_path=file_path,
                block_size=512
    )

    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm
    )

    return data_collator


def postprocess_text(preds, labels):
    # TODO: Separate only the subject from string
    # Ensure that for preds, you have a list of only the generated subject parts
    # For labels, it should be a list of list of only the reference subjects
    # NO OTHER CONTENT: EMAIL / SEPARATORS SHOULD BE OUTPUT AFTER POSTPROCESSING

    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    # print('logits:', logits.shape)
    pred_ids = torch.argmax(logits, dim=-1)
    # print('pred_ids:', pred_ids.shape)

    return pred_ids, labels


def compute_metrics(eval_preds):
    list_of_prediction = []
    for i in range(0, len(df)):
#               print(i)
#               print(list_of_references[i])
              inputs = tokenizer("Question: " + df.iloc[i, 0] + ' Answer: ', return_tensors="pt")
              inputs['input_ids'] = inputs['input_ids'].cpu()  # Move input tensor to CPU if necessary
              device = torch.device("cuda:0")  # Specify the CUDA device
              model.to(device)  # Move the model to the CUDA device

              # Move the input tensor to the CUDA device
              inputs['input_ids'] = inputs['input_ids'].to(device)
              outputs = model.generate(inputs['input_ids'], max_new_tokens=30, do_sample=True, top_k=30, top_p=0.95)
              prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
              # Generate outputs using the model on the CUDA device
              #print(prediction)
              prediction = prediction.split('Answer: ')[1]
              list_of_prediction.append(prediction)
#               print(prediction)
    result = rouge.compute(predictions=list_of_prediction, references=list_of_references)
    results_sacrebleu = sacrebleu.compute(predictions=list_of_prediction, references=list_of_references, lowercase = True)
    #results_meteor = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    results_bert = bertscore.compute(predictions=list_of_prediction, references=list_of_references,lang="en")
    wandb.log({
        "R1": round(result["rouge1"], 4),
        "R2": round(result["rouge2"], 4),
        "RL": round(result["rougeL"], 4),
        "RLsum": round(result["rougeLsum"], 4),
        "bleu": round(results_sacrebleu["score"], 4)
    })
    return {
        "R1": round(result["rouge1"], 4),
        "R2": round(result["rouge2"], 4),
        "RL": round(result["rougeL"], 4),
        "RLsum": round(result["rougeLsum"], 4),
        "bleu": round(results_sacrebleu["score"], 4),
        "precision1":round(results_bert["precision"][0], 4),
        "precision2":round(results_bert["precision"][1], 4),
        "recall1":round(results_bert["recall"][0], 4),
        "recall2":round(results_bert["recall"][1], 4),
        "f1-score1":round(results_bert["f1"][0], 4),
        "f1-score2":round(results_bert["f1"][1],4)
    }

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model.save_pretrained(output_dir)

train_dataset = load_dataset(train_file_path, tokenizer)
eval_dataset = load_dataset(eval_file_path, tokenizer)
data_collator = load_data_collator(tokenizer)

tokenizer.save_pretrained(output_dir)

training_args = TrainingArguments(
          output_dir='/content/result/',
          evaluation_strategy = "epoch",
#          eval_steps = 500,
          learning_rate=1e-5,
#          save_strategy = "epoch",
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          per_device_eval_batch_size=1,
          num_train_epochs=num_train_epochs
      )

trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          preprocess_logits_for_metrics=preprocess_logits_for_metrics,
          compute_metrics=compute_metrics
)
with wandb.init(project="team6"):
  trainer.train()
#trainer.save_model()

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,R1,R2,Rl,Rlsum,Bleu,Precision1,Precision2,Recall1,Recall2,F1-score1,F1-score2
1,No log,4.468911,0.185400,0.058400,0.145200,0.144000,2.999000,0.000000,0.855300,0.000000,0.842700,0.000000,0.845200
2,5.834800,4.677147,0.184100,0.055000,0.143200,0.142000,4.268000,0.968300,0.869300,0.955900,0.867400,0.962000,0.868400
3,2.943600,4.804050,0.184700,0.055700,0.140400,0.139600,3.572000,0.936200,0.846200,0.925800,0.855200,0.930900,0.850700
4,2.741600,4.854162,0.234000,0.067600,0.189000,0.187700,4.874400,0.929600,0.881400,0.944500,0.880600,0.935600,0.881000
5,2.595200,4.911634,0.250900,0.083400,0.197800,0.198100,6.200200,0.865200,0.870800,0.870000,0.866700,0.867200,0.863200
6,2.502200,5.005701,0.270600,0.084100,0.211200,0.209700,7.005100,0.912300,0.852500,0.911600,0.852900,0.911900,0.852700
7,2.421100,5.053088,0.264500,0.088900,0.205500,0.204700,6.833600,0.901100,0.860000,0.897100,0.853200,0.899100,0.852600
8,2.364000,5.170439,0.281500,0.102700,0.220400,0.220800,8.765000,0.873400,0.873500,0.882800,0.863600,0.878100,0.865400
9,2.364000,5.219799,0.283000,0.097800,0.220400,0.219300,7.305600,0.840800,0.874700,0.845300,0.859100,0.843000,0.866800
10,2.322500,5.232075,0.296500,0.098800,0.230700,0.229700,7.571500,0.897200,0.879000,0.895400,0.856700,0.896300,0.867700


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `

R1,▁▁▁▄▅▆▆▇▇█
R2,▁▁▁▃▅▅▆█▇▇
RL,▁▁▁▅▅▆▆▇▇█
RLsum,▁▁▁▅▆▆▆▇▇█
bleu,▁▃▂▃▅▆▆█▆▇
eval/R1,▁▁▁▄▅▆▆▇▇█
eval/R2,▁▁▁▃▅▅▆█▇▇
eval/RL,▁▁▁▅▅▆▆▇▇█
eval/RLsum,▁▁▁▅▆▆▆▇▇█
eval/bleu,▁▃▂▃▅▆▆█▆▇
eval/f1-score1,▁███▇██▇▇█
